
# 🔍 AI-Generated vs. Real Image Classifier (Colab Starter)

This notebook trains a simple baseline classifier to distinguish **AI-generated** images from **real** images using transfer learning (EfficientNetB0) in Keras. It includes:
- GPU setup check
- Two data ingestion options (Google Drive folder or Kaggle download)
- Data augmentation
- Transfer learning (frozen base → fine-tune)
- Metrics & confusion matrix
- **Grad-CAM** visualizations for "why the model thinks so"

> **Folder structure expected (if using Drive):**
>
> ```
> /MyDrive/datasets/aivsreal/
>   train/
>     ai/
>     real/
>   val/
>     ai/
>     real/
>   test/
>     ai/
>     real/
> ```
>
> Upload dataset here.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 0) Runtime setup

In [2]:

# If using Colab, set: Runtime → Change runtime type → GPU
import tensorflow as tf
print("TensorFlow:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))


TensorFlow: 2.19.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 1) Mount Google Drive (recommended)

In [3]:

# 👇 Change this path to where your dataset lives in Drive
DATA_DIR = "/content/drive/MyDrive/Datasets/aivsreal"
print("Using DATA_DIR:", DATA_DIR)


Using DATA_DIR: /content/drive/MyDrive/Datasets/aivsreal



## (Optional) 1b) Download a dataset via Kaggle

If you have a Kaggle dataset you'd like to use, do this once per session:
1. Create an API token at https://www.kaggle.com/settings/account (click "Create New API Token").
2. Upload the downloaded `kaggle.json` using the cell below.
3. Replace the example dataset path with your chosen dataset slug.

> Tip: Many datasets already have `train/val/test` or `train/test`. If not, you can split after download.


In [4]:

# Uncomment to use Kaggle (run step-by-step):
# from google.colab import files
# files.upload()  # upload kaggle.json
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# EXAMPLE: Replace with your dataset (this is a placeholder; change to a real one you select)
# !kaggle datasets download -d <owner>/<dataset-slug> -p /content/data
# !unzip -q /content/data/*.zip -d /content/data

# After unzip, set DATA_DIR to the folder that contains class subfolders (ai/ real/)
# DATA_DIR = "/content/data/<folder_with_class_subdirs>"


## 2) Build datasets

In [5]:

import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 1337

# Helper to load a split if it exists; else return None
def try_load_dataset(split_name):
    split_dir = os.path.join(DATA_DIR, split_name)
    if os.path.isdir(split_dir):
        ds = keras.preprocessing.image_dataset_from_directory(
            split_dir,
            image_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            seed=SEED,
            label_mode='categorical'  # 2 classes: [ai, real]
        )
        return ds
    return None

train_ds = try_load_dataset("train")
val_ds   = try_load_dataset("val")
test_ds  = try_load_dataset("test")

# If no /val, split from train
if train_ds is not None and val_ds is None:
    val_size = 0.2
    train_ds = keras.preprocessing.image_dataset_from_directory(
        os.path.join(DATA_DIR, "train"),
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        validation_split=val_size,
        subset="training",
        seed=SEED,
        label_mode='categorical'
    )
    val_ds = keras.preprocessing.image_dataset_from_directory(
        os.path.join(DATA_DIR, "train"),
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        validation_split=val_size,
        subset="validation",
        seed=SEED,
        label_mode='categorical'
    )

if train_ds is None:
    raise SystemExit("❌ Could not find a 'train/' folder inside DATA_DIR. See the expected structure above.")

# Cache+prefetch
AUTOTUNE = tf.data.AUTOTUNE
def configure(ds):
    return ds.cache().prefetch(buffer_size=AUTOTUNE)

train_ds = configure(train_ds)
val_ds   = configure(val_ds) if val_ds is not None else None
test_ds  = configure(test_ds) if test_ds is not None else None

class_names = train_ds.class_names
print("Classes:", class_names)


Found 0 files belonging to 2 classes.


ValueError: No images found in directory /content/drive/MyDrive/Datasets/aivsreal/train. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

## 3) Data augmentation

In [ ]:

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
], name="augmentation")


## 4) Build the model (EfficientNetB0)

In [ ]:

base = keras.applications.EfficientNetB0(
    include_top=False, input_shape=IMG_SIZE + (3,), weights="imagenet"
)
base.trainable = False  # freeze for initial training

inputs = keras.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = keras.applications.efficientnet.preprocess_input(x)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()


## 5) Train (Stage 1: frozen base)

In [ ]:

EPOCHS_1 = 5  # You can increase later
history1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_1
)


## 6) Fine-tune (unfreeze top of base)

In [ ]:

# Unfreeze top blocks for gentle fine-tuning
base.trainable = True
for layer in base.layers[:-40]:  # unfreeze last ~40 layers
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

EPOCHS_2 = 5  # You can increase later
history2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_2
)


## 7) Evaluate and Confusion Matrix

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# Pick test_ds if available, else use val_ds for quick eval
eval_ds = test_ds if test_ds is not None else val_ds

if eval_ds is None:
    raise SystemExit("❌ No validation or test set found. Please create a 'val/' or 'test/' split.")

y_true = np.concatenate([y.numpy() for _, y in eval_ds], axis=0)
y_true_lbl = np.argmax(y_true, axis=1)

y_pred_prob = model.predict(eval_ds)
y_pred_lbl = np.argmax(y_pred_prob, axis=1)

print(classification_report(y_true_lbl, y_pred_lbl, target_names=class_names))

cm = confusion_matrix(y_true_lbl, y_pred_lbl, normalize=None)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
plt.figure(figsize=(5,5))
disp.plot(values_format='d')
plt.title("Confusion Matrix")
plt.show()


## 8) Grad-CAM (Why the model thinks so)

In [ ]:

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

# Find last conv layer name in EfficientNetB0 backbone
last_conv = None
for layer in model.layers:
    if hasattr(layer, "name") and "efficientnetb0" in layer.name:
        # grab internal conv name via the base model
        for l in layer.layers[::-1]:
            if isinstance(l, tf.keras.layers.Conv2D):
                last_conv = l.name
                break
        break

print("Using last conv layer:", last_conv)

def show_gradcam(dataset, n=4):
    it = dataset.unbatch().take(n)
    for img, label in it:
        img_np = tf.image.resize(img, IMG_SIZE).numpy()
        img_batch = np.expand_dims(img_np, axis=0)
        heatmap = make_gradcam_heatmap(img_batch, model, last_conv)
        # overlay
        heatmap_resized = tf.image.resize(heatmap[..., np.newaxis], IMG_SIZE).numpy().squeeze()
        plt.figure(figsize=(8,3))
        plt.subplot(1,3,1); plt.imshow(img_np.astype("uint8")); plt.axis('off'); plt.title("Image")
        plt.subplot(1,3,2); plt.imshow(heatmap_resized, cmap='jet'); plt.axis('off'); plt.title("Grad-CAM")
        # overlay
        overlay = img_np.astype("float32")
        overlay = (0.4 * overlay + 0.6 * (plt.cm.jet(heatmap_resized)[..., :3] * 255)).clip(0,255).astype("uint8")
        plt.subplot(1,3,3); plt.imshow(overlay); plt.axis('off'); plt.title("Overlay")
        plt.show()

# Visualize a few
show_gradcam(eval_ds, n=4)


## 9) Save model

In [ ]:

SAVE_DIR = "/content/drive/MyDrive/models/aivsreal_baseline"
os.makedirs(SAVE_DIR, exist_ok=True)
model.save(os.path.join(SAVE_DIR, "effnetb0_aivsreal.h5"))
print("Saved to", SAVE_DIR)



## ✅ Next steps
- Add **more data** and balance classes.
- Try stronger backbones (e.g., `EfficientNetB3`, `ResNet50`, or `ViT` via `keras_cv`).
- Add **augmentations** targeted at AI artifacts (e.g., JPEG re-compression, blur, noise) to improve robustness.
- Run **k-fold cross-validation** for reliable metrics.
- Log to **Weights & Biases** or **TensorBoard** for experiment tracking.
- Consider **frequency-domain features** or **patch-level** training as enhancements.
